In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
import time
from selenium.webdriver.chrome.service import Service
import os


In [50]:
import os
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    StaleElementReferenceException,
    TimeoutException,
)
from selenium.webdriver.chrome.service import Service

# =======================
# ★ 你要自己設定的參數 ★
# =======================

# 有「Export」按鈕的那一頁（不是 overlay 的 URL）
BASE_URL = "https://www.webofscience.com/wos/woscc/summary/e5e64c73-cc6f-4d8f-9f54-37d62495a2f3-018a0b4503/relevance/1"

# 總筆數 n（寫死即可，例如 7265）
TOTAL_N = 100000
# 每次最多匯出 1000 筆（WOS 限制）
BATCH_SIZE = 1000

# Windows 預設下載路徑（你可以依需要改掉）
DOWNLOAD_DIR = os.path.join(os.path.expanduser("~"), "Downloads")
# 或直接寫死：
# DOWNLOAD_DIR = r"C:\Users\techang\Downloads"

# 每個 batch 完成後的休息秒數（避免太頻繁）
WAIT_BETWEEN_BATCH = 3

print("55555555")

# =======================
# 共用工具函式
# =======================
def click_export_and_wait_for_start(driver, locator, desc="", timeout=30):
    """
    只點一次 Export，然後等待 .crdownload 出現。
    不會 retry，也不會第二次點擊。
    """
    print(f"➡️ 點擊 {desc}（只點一次）")

    # 第一次（也是唯一一次）點擊
    try:
        elem = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable(locator)
        )
        driver.execute_script("arguments[0].click();", elem)
    except Exception as e:
        print(f"⚠️ 點擊 {desc} 失敗：{e}")
        return False

    print("⏳ 已按下 Export，等待下載開始 (.crdownload 出現)…")

    start_time = time.time()

    # 等 .crdownload 出現
    while time.time() - start_time < timeout:
        cr_files = [f for f in os.listdir(DOWNLOAD_DIR)
                    if f.lower().endswith(".crdownload")]
        if cr_files:
            print(f"🟩 下載已開始（偵測到 .crdownload: {cr_files}）")
            return True

        time.sleep(1)

    print("❌ 等待超過 timeout，但仍未看到下載開始")
    return False

def click_export_and_confirm(driver, locator, desc="", timeout=10):
    """
    替代 js_click，只用在『彈窗 Export 按鈕』。
    - 按一次 Export
    - 檢查是否開始下載（.crdownload 是否出現）
    - 若已開始下載 → 不再按第二次（避免重複下載）
    - 若未開始下載 → 再按一次（最多兩次）
    """
    # 按第一下
    print(f"➡️ 點擊 {desc}（第 1 次）")
    try:
        elem = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable(locator)
        )
        driver.execute_script("arguments[0].click();", elem)
    except Exception as e:
        print(f"⚠️ {desc} 點擊失敗：{e}")

    time.sleep(1)

    # 檢查是否開始下載
    cr_files = [f for f in os.listdir(DOWNLOAD_DIR)
                if f.lower().endswith(".crdownload")]
    if cr_files:
        print(f"🟩 下載已開始（{cr_files}）")
        return True

    print("⚠️ 沒看到下載開始 → 再按一次 Export")

    # 按第二次（最後一次）
    try:
        elem = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable(locator)
        )
        driver.execute_script("arguments[0].click();", elem)
    except Exception as e:
        print(f"⚠️ 第二次點擊 {desc} 失敗：{e}")

    time.sleep(1)

    # 再檢查一次
    cr_files = [f for f in os.listdir(DOWNLOAD_DIR)
                if f.lower().endswith(".crdownload")]
    if cr_files:
        print(f"🟩 第二次點擊成功 → 下載開始（{cr_files}）")
        return True

    print("❌ Export 按鈕點擊後仍未開始下載")
    return False


def wait_for_downloads(download_dir, timeout=120):
    """
    等待下載完成：只要資料夾裡還有 .crdownload，就持續等待。
    timeout 秒還沒完成就回傳 False。
    """
    start = time.time()
    while True:
        tmp_files = [f for f in os.listdir(download_dir)
                     if f.lower().endswith(".crdownload")]
        if not tmp_files:
            print("✅ 下載完成，沒有 .crdownload 檔了")
            return True
        if time.time() - start > timeout:
            print("⚠️ 等待下載超過 timeout，仍有 .crdownload：", tmp_files)
            return False
        print("⏳ 下載中，偵測到 .crdownload：", tmp_files, "，再等 2 秒...")
        time.sleep(2)

def rename_last_download(start_rec, end_rec):
    """
    在 .crdownload 消失後，等待新生成的 Excel 檔案，
    並確保不會把上一批改成下一批名稱。
    """
    before_files = set(f for f in os.listdir(DOWNLOAD_DIR)
                       if f.lower().endswith((".xls", ".xlsx")))

    # 最多等 10 秒讓新檔案出現
    wait_time = 0
    new_file = None

    while wait_time < 10:
        now_files = set(f for f in os.listdir(DOWNLOAD_DIR)
                        if f.lower().endswith((".xls", ".xlsx")))

        # 找「本來沒有，現在才出現的檔案」＝最新下載的
        diff = now_files - before_files
        if diff:
            # 只會有 1 個
            new_file = diff.pop()
            break

        wait_time += 1
        time.sleep(1)

    if not new_file:
        print("❌ 找不到新下載的 Excel，無法重新命名")
        return

    # rename
    old_path = os.path.join(DOWNLOAD_DIR, new_file)
    ext = ".xlsx" if new_file.endswith(".xlsx") else ".xls"

    new_name = f"{start_rec}_{end_rec}{ext}"
    new_path = os.path.join(DOWNLOAD_DIR, new_name)

    try:
        os.rename(old_path, new_path)
        print(f"🟩 已重新命名：{new_file} → {new_name}")
    except Exception as e:
        print(f"⚠️ 檔案重新命名失敗：{e}")


def js_click(driver, locator, desc=""):
    """
    等元素可以被點擊，然後用 JS 方式點擊。
    這個函式「只負責點擊」，不處理輸入值。
    """
    for attempt in range(3):
        try:
            print(f"➡️ ({attempt+1}/3) 嘗試點擊 {desc}：{locator}")
            elem = WebDriverWait(driver, 15).until(
                EC.presence_of_element_located(locator)
            )
            driver.execute_script(
                "arguments[0].scrollIntoView({block: 'center'});", elem
            )
            time.sleep(0.2)
            driver.execute_script("arguments[0].click();", elem)
            return
        except StaleElementReferenceException:
            print(f"⚠️ {desc} 元素 stale，重試中...")
            time.sleep(0.5)
    raise StaleElementReferenceException(f"多次點擊失敗：{desc} - {locator}")


def set_input_value(driver, locator, value, desc=""):
    """
    填 input 用的安全函式。
    """
    for attempt in range(3):
        try:
            print(f"➡️ ({attempt+1}/3) 準備填寫 {desc}，值 = {value}")
            elem = WebDriverWait(driver, 15).until(
                EC.visibility_of_element_located(locator)
            )
            driver.execute_script(
                "arguments[0].scrollIntoView({block: 'center'});", elem
            )
            time.sleep(0.2)
            elem.clear()
            elem.send_keys(str(value))
            print(f"✅ 已填寫 {desc}")
            return
        except StaleElementReferenceException:
            print(f"⚠️ {desc} 元素 stale，重試中...")
            time.sleep(0.5)
    raise StaleElementReferenceException(f"多次填寫失敗（stale）：{desc} - {locator}")


def accept_cookies_if_needed(driver):
    """如果有 cookie / 同意 視窗，就把它點掉，沒有就略過。"""
    try:
        cookie_btn = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((
                By.XPATH,
                "//button[contains(., 'Accept') or "
                "contains(., '同意') or contains(., '接受') or "
                "contains(., '我知道了') or contains(., 'Got it')]"
            ))
        )
        driver.execute_script("arguments[0].click();", cookie_btn)
        print("✅ 已點擊 Cookie 視窗")
    except TimeoutException:
        print("ℹ️ 沒偵測到 Cookie 視窗（或文字不同），略過")


# =======================
# 單次「匯出一個區間」的流程
# =======================

def export_range(driver, wait, start_rec, end_rec):
    """
    在「目前這一頁」執行一次 Export：
    匯出 [start_rec, end_rec] 的紀錄。
    """
    print(f"\n=== ▶️ 開始匯出 {start_rec} ~ {end_rec} ===")

    # 1️⃣ 點上方工具列的「Export」主按鈕
    js_click(driver, (By.ID, "export-trigger-btn"), "Export 主按鈕")

    # 2️⃣ 在 dropdown 裡點「Excel」
    js_click(driver, (By.ID, "exportToExcelButton"), "Excel 選項")

    # 3️⃣ 選「Records from」那顆 radio（id="radio3-input」）
    js_click(driver, (By.ID, "radio3-input"), "Records from radio")

    # 4️⃣ 填 from / to（用安全函式）
    set_input_value(
        driver,
        (By.CSS_SELECTOR, "input[name='markFrom']"),
        start_rec,
        desc="from (markFrom)",
    )
    set_input_value(
        driver,
        (By.CSS_SELECTOR, "input[name='markTo']"),
        end_rec,
        desc="to (markTo)",
    )

    # 5️⃣ Record Content → 選「Full Record」
    js_click(
        driver,
        (By.CSS_SELECTOR, "wos-select button.dropdown"),
        "Record Content 下拉選單",
    )
    js_click(
        driver,
        (
            By.XPATH,
            "//div[@role='menuitem']//span[normalize-space()='Full Record']",
        ),
        "Full Record 選項",
    )

    # 6️⃣ 按彈窗右下角的「Export」按鈕（加強確認下載有開始）
    #click_export_and_wait_for_start(driver, (By.ID, "exportButton"), "彈窗 Export 按鈕")
    # 6️⃣ 按彈窗右下角的「Export」按鈕
    js_click(driver, (By.ID, "exportButton"), "彈窗 Export 按鈕")

# 等下載完成
    

    #time.sleep(3)

    # 7️⃣ 等下載真正完成（看 Downloads 裡有沒有 .crdownload）
    print("⏳ 等待這一批下載完成（偵測 .crdownload）...")
    ok = wait_for_downloads(DOWNLOAD_DIR, timeout=180)
    if not ok:
        print("⚠️ 下載等待超時，可能有檔案沒抓完，建議手動檢查一次。")
    else:
        print("✅ 這一批匯出完成！")
        rename_last_download(start_rec, end_rec)  # ⭐ 重新命名


# =======================
# 主程式：自動分批
# =======================

print("55555555")

def main():


    options = webdriver.ChromeOptions()

    # ⭐ 不要跳多檔下載允許視窗
    prefs = {
        "profile.default_content_setting_values.automatic_downloads": 1,
    }
    options.add_experimental_option("prefs", prefs)

    service = Service(r"C:\tools\chromedriver\chromedriver-win64\chromedriver.exe")
    driver = webdriver.Chrome(service=service, options=options)

    wait = WebDriverWait(driver, 15)


    try:
        print("➡️ 開啟列表頁...")
        driver.get(BASE_URL)
        driver.maximize_window()

        # 先處理 Cookie
        accept_cookies_if_needed(driver)

        # 依照 TOTAL_N 自動切批
        print(f"\n🔢 總筆數：{TOTAL_N}，每批 {BATCH_SIZE} 筆，開始自動分批匯出...")
        batch_idx = 0

        for start in range(50001, TOTAL_N + 1, BATCH_SIZE):
            batch_idx += 1
            end = min(start + BATCH_SIZE - 1, TOTAL_N)

            print(f"\n==== 🧾 Batch {batch_idx}: {start} ~ {end} ====")
            try:
                export_range(driver, wait, start, end)
            except TimeoutException as e:
                print(f"⚠️ Batch {batch_idx} 匯出失敗（Timeout）：{e}")
            except StaleElementReferenceException as e:
                print(f"⚠️ Batch {batch_idx} 匯出失敗（Stale）：{e}")

            # 每批之間再多休息一下
            print(f"😴 批次間休息 {WAIT_BETWEEN_BATCH} 秒避免太頻繁...")
            time.sleep(WAIT_BETWEEN_BATCH)

        print("\n🎉 全部批次匯出完成！")

    finally:
        #driver.quit()
        pass

print("55555555")

if __name__ == "__main__":
    main()


55555555
55555555
55555555
➡️ 開啟列表頁...
ℹ️ 沒偵測到 Cookie 視窗（或文字不同），略過

🔢 總筆數：100000，每批 1000 筆，開始自動分批匯出...

==== 🧾 Batch 1: 50001 ~ 51000 ====

=== ▶️ 開始匯出 50001 ~ 51000 ===
➡️ (1/3) 嘗試點擊 Export 主按鈕：('id', 'export-trigger-btn')
➡️ (1/3) 嘗試點擊 Excel 選項：('id', 'exportToExcelButton')
➡️ (1/3) 嘗試點擊 Records from radio：('id', 'radio3-input')
➡️ (1/3) 準備填寫 from (markFrom)，值 = 50001
✅ 已填寫 from (markFrom)
➡️ (1/3) 準備填寫 to (markTo)，值 = 51000
✅ 已填寫 to (markTo)
➡️ (1/3) 嘗試點擊 Record Content 下拉選單：('css selector', 'wos-select button.dropdown')
➡️ (1/3) 嘗試點擊 Full Record 選項：('xpath', "//div[@role='menuitem']//span[normalize-space()='Full Record']")
➡️ (1/3) 嘗試點擊 彈窗 Export 按鈕：('id', 'exportButton')
⏳ 等待這一批下載完成（偵測 .crdownload）...
✅ 下載完成，沒有 .crdownload 檔了
✅ 這一批匯出完成！
🟩 已重新命名：savedrecs.xls → 50001_51000.xls
😴 批次間休息 3 秒避免太頻繁...

==== 🧾 Batch 2: 51001 ~ 52000 ====

=== ▶️ 開始匯出 51001 ~ 52000 ===
➡️ (1/3) 嘗試點擊 Export 主按鈕：('id', 'export-trigger-btn')
➡️ (1/3) 嘗試點擊 Excel 選項：('id', 'exportToExcelButton')
➡️ (1/3) 嘗試點擊 